<a href="https://colab.research.google.com/github/wakristensen/machine-learning-workshop/blob/main/04_supervised_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1) Importer biblioteker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TensorFlow:", tf.__version__)


In [ ]:
# --- Produktfokusert datasett for antall salg ---
file_id = "1Ku-y5BE5CdGQwzEMQ491cfIYWILS6T8U"
url = f"https://drive.google.com/uc?id={file_id}"

df = pd.read_csv(url, encoding="latin1")
# Sikre riktig dtype
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Fjerner kansellerte og negative linjer, slik at "antall salg" kun betyr positive salg
# Kansellerte transaksjoner har InvoiceNo som starter med 'C', negativ Quantity, og samme UnitPrice som originalen
df_prod = df.copy()

# Finne kansellerte transaksjoner
cancelled_transactions = df_prod[df_prod['InvoiceNo'].astype(str).str.startswith('C')].copy()

# Lager en unik nøkkel for å matche kansellerte transaksjoner med originale counterparts
cancelled_transactions['match_key'] = cancelled_transactions['StockCode'].astype(str) + '_' + \
                                      cancelled_transactions['CustomerID'].astype(str) + '_' + \
                                      (cancelled_transactions['Quantity'] * -1).astype(str) + '_' + \
                                      cancelled_transactions['UnitPrice'].astype(str)

# Finner originale transaksjoner som matcher de kansellerte
df_prod['match_key'] = df_prod['StockCode'].astype(str) + '_' + \
                       df_prod['CustomerID'].astype(str) + '_' + \
                       df_prod['Quantity'].astype(str) + '_' + \
                       df_prod['UnitPrice'].astype(str)

# Finner transaksjoner vi skal fjerne (kansellerte transaksjoner og deres positive counterparts)
keys_to_remove = cancelled_transactions['match_key'].tolist()

# Filtrerer vekk transaksjoner vi skal fjerne
df_prod = df_prod[~df_prod['match_key'].isin(keys_to_remove)].copy()

# Fjerner den midlertidig match-nøkkelen
df_prod = df_prod.drop(columns=['match_key'])

# Sikrer at utelukkende positive kvantiteter og unit priser står igje
df_prod = df_prod[(df_prod['Quantity'] > 0) & (df_prod['UnitPrice'] > 0)]


# Topp 10 etter frekvens av Description
top_products = df_prod['Description'].value_counts().head(10)
print("Topp 10 produkter:", top_products)

# Velg mest frekvente beskrivelse
TOP_PRODUCT = df_prod['Description'].value_counts().reset_index().iloc[2][0]
print("Produkt valgt (Description):", TOP_PRODUCT)

# Filtrer på Description, ikke StockCode
prod_df = df_prod[df_prod['Description'] == TOP_PRODUCT].copy()

# Sikre datetime og lag eksplisitt Date-kolonne
prod_df['InvoiceDate'] = pd.to_datetime(prod_df['InvoiceDate'], errors='coerce')
prod_df = prod_df.dropna(subset=['InvoiceDate'])

# Behold dtype datetime64 ved å normalisere til midnatt (bedre for plotting og sortering)
prod_df['Date'] = prod_df['InvoiceDate'].dt.normalize()

# Gruppér per dag og summer Quantity
daily_prod_sales = (
    prod_df
    .groupby('Date', as_index=False)['Quantity']
    .sum()
    .rename(columns={'Quantity': 'SalesCount'})
    .sort_values('Date')
)

print("Form på daglig datasett:", daily_prod_sales.shape)
daily_prod_sales

In [ ]:
# KORT EDA
plt.figure(figsize=(12,4))
sns.lineplot(data=daily_prod_sales, x='Date', y='SalesCount')
plt.title(f'Daglig antall salg, produkt {TOP_PRODUCT}')
plt.xlabel('Dato'); plt.ylabel('Antall')
plt.grid(True); plt.tight_layout(); plt.show()

# Glidende snitt
daily_prod_sales['Rolling7'] = daily_prod_sales['SalesCount'].rolling(7).mean()
plt.figure(figsize=(12,4))
sns.lineplot(data=daily_prod_sales, x='Date', y='SalesCount', label='Faktisk')
sns.lineplot(data=daily_prod_sales, x='Date', y='Rolling7', label='Rolling7')
plt.title('Daglig salg med 7-dagers rullende snitt')
plt.xlabel('Dato'); plt.ylabel('Antall')
plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()


In [ ]:
def make_prod_features(df):
    df = df.copy()
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['dayofweek'] = df['Date'].dt.dayofweek  # 0 mandag

    # Enkle lag og rolling
    df['lag_1'] = df['SalesCount'].shift(1)
    df['lag_7'] = df['SalesCount'].shift(7)
    df['roll_mean_7'] = df['SalesCount'].rolling(7).mean()

    # Dropp NaN som følger av lag og rolling
    df = df.dropna().reset_index(drop=True)

    feature_cols = ['year', 'month', 'dayofweek', 'lag_1', 'lag_7', 'roll_mean_7']
    X = df[feature_cols]
    y = df['SalesCount']
    return df, X, y, feature_cols

features_df, X, y, feature_cols = make_prod_features(daily_prod_sales)
features_df.head()


In [ ]:
# Del etter dato, ikke shuffle
cutoff = features_df['Date'].max() - pd.DateOffset(days=45)
train_mask = features_df['Date'] <= cutoff
test_mask  = features_df['Date'] >  cutoff

X_train, y_train = X[train_mask], y[train_mask]
X_test,  y_test  = X[test_mask],  y[test_mask]

# Skaler features (NN liker skalerte data)
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc  = scaler.transform(X_test)

X_train.shape, X_test.shape


In [ ]:
tf.keras.backend.clear_session()

# Enkel MLP
model = keras.Sequential([
    layers.Input(shape=(X_train_sc.shape[1],)), #Definerer input formen (antall features i X)
    layers.Dense(64, activation='relu'), # Et fullkoblet lag med 64 noder, ReLU som aktiveringsfunksjon
    layers.Dense(32, activation='relu'), # Nytt lag med 32 noder
    layers.Dense(1)  # regresjon - outputlaget med 1 node, siden vi predikerer en kontinuerlig verdi (Antall salg), ingen aktivering herm siden det er regresjon
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), # Hvordan modellen skal lære (backpropagation) Adam er en populær gradient descent-algoritme
              loss='mse', # Kostnadsfunksjonen, mean squared error, vanlig for regresjon
              metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])  # Overvåker RMSE under trening og validering

callbacks = [ #Callbacks er små hjelpere som styrer trening underveis
    # Stopper treningen hvis validerings-RMSE ikke blir bedre på 10 epoker. Restore_best_weights gjør at modellen går tilbake til vektene som ga best resultat
    keras.callbacks.EarlyStopping(monitor='val_rmse', patience=10, restore_best_weights=True),
    # ReduceLROnPlateu halverer læringsraten hvis validerings-RMSE ikke forbedres på 5 epoker. Det hjelper modellen med "finpussing" når forbedringen flater ut
    keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=5, verbose=0)
]

history = model.fit(
    X_train_sc, y_train, # Treningsdata
    validation_data=(X_test_sc, y_test), # Brukes til å måle ytelse underveis
    epochs=200, # Maks antall gjennomganger av treningssettet, kan stoppe tidligere pga earlystopping
    batch_size=50, # Antall observasjoner i hver batch, delmengde av data som oppdaterer vektene
    callbacks=callbacks, # Kobler inn earlystopping og reduce on plateau
    verbose=0 # Undertrykker logging i output
)

print("Beste val-RMSE:", np.min(history.history['val_rmse']))


In [ ]:
# Prediksjoner
y_pred_nn = model.predict(X_test_sc).ravel()

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

mae = mean_absolute_error(y_test, y_pred_nn)
rmse_val = rmse(y_test, y_pred_nn)
r2 = r2_score(y_test, y_pred_nn)

print(f"NN -> MAE: {mae:.2f} | RMSE: {rmse_val:.2f} | R²: {r2:.3f}")

# Plot faktisk vs predikert på dato
test_dates = features_df.loc[test_mask, 'Date'].reset_index(drop=True)

plt.figure(figsize=(12,5))
plt.plot(test_dates, y_test.values, label='Faktisk')
plt.plot(test_dates, y_pred_nn, 'r--', label='Predikert (NN)')
plt.title('Faktisk vs predikert i hold‑out perioden')
plt.xlabel('Dato'); plt.ylabel('Antall salg')
plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

# Scatter: faktisk vs predikert
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_nn, alpha=0.6)
lims = [min(y_test.min(), y_pred_nn.min()), max(y_test.max(), y_pred_nn.max())]
plt.plot(lims, lims, 'r--')
plt.xlabel('Faktisk'); plt.ylabel('Predikert')
plt.title('Faktisk vs predikert (NN)')
plt.grid(True); plt.tight_layout(); plt.show()


In [ ]:
# Liten baseline-sjekk for sammenligning
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train_sc, y_train)
y_pred_lin = linreg.predict(X_test_sc)

mae_lr  = mean_absolute_error(y_test, y_pred_lin)
rmse_lr = rmse(y_test, y_pred_lin)
r2_lr   = r2_score(y_test, y_pred_lin)

print(f"LR  -> MAE: {mae_lr:.2f} | RMSE: {rmse_lr:.2f} | R²: {r2_lr:.3f}")
print(f"NN  -> MAE: {mae:.2f} | RMSE: {rmse_val:.2f} | R²: {r2:.3f}")


## Ble NN bedre enn baseline?
1) Sammenlign MAE, RMSE og R² for NN og Linear Regression.
2) Prøv å forbedre NN litt:
   - Øk antall noder i første lag (for eksempel 128).
   - Legg til Dropout(0.2) mellom lagene.
   - Endre læringsraten i Adam fra 1e-3 til 5e-4.
   - Kjør noen få runder til og se om validerings-RMSE går ned.

Refleksjon:
- Hva skjer med valideringskurven når du øker modellen?
- Overtrener modellen raskere?
- Hva ville du gjort for å stabilisere treningen (flere data, enklere features, annen hold‑out)?


In [ ]:
# Bonus: litt større modell
tf.keras.backend.clear_session()

big_model = keras.Sequential([
    layers.Input(shape=(X_train_sc.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

big_model.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-4),
                  loss='mse',
                  metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])

history2 = big_model.fit(
    X_train_sc, y_train,
    validation_data=(X_test_sc, y_test),
    epochs=250,
    batch_size=32,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_rmse', patience=12, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=6, verbose=0)
    ],
    verbose=0
)

y_pred_big = big_model.predict(X_test_sc).ravel()
print("Bonus NN -> MAE:", mean_absolute_error(y_test, y_pred_big),
      "| RMSE:", rmse(y_test, y_pred_big),
      "| R²:", r2_score(y_test, y_pred_big))
